In [2]:
import sys
import os
import requests

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.ingestion.auth import strava_auth

In [ ]:
# TODO: Add short strava docs
# https://developers.strava.com/docs/reference/#api-Athletes

In [ ]:
athlete_url = 'https://www.strava.com/api/v3/athlete'
access_token = strava_auth.get_access_token()
headers = {'Authorization': f'Bearer {access_token}'}

137549 1184ee4b53c17a4caec26f1fe6b1f0169d1136a4 c7b0516c8801fde1965ea36e40428b81941ba448
Refreshing Strava access token...
Successfully refreshed access token.


# General athlete information

In [5]:
import requests, datetime as dt

def get_debug(url, headers, **kwargs):
    try:
        r = requests.get(url, headers=headers, timeout=10, **kwargs)
        r.raise_for_status()
        print("OK", r.status_code)
        return r
    except requests.HTTPError as e:
        r = e.response
        hdrs = {k: r.headers.get(k) for k in [
            "X-RateLimit-Limit", "X-RateLimit-Usage",
            "X-ReadRateLimit-Limit", "X-ReadRateLimit-Usage",
            "Retry-After", "Date"
        ]}
        print("HTTPError:", r.status_code)
        print("Headers:", hdrs)
        print("Body (first 300):", r.text[:300])
        raise

athlete_url = "https://www.strava.com/api/v3/athlete"
resp = get_debug(athlete_url, headers=headers)
data = resp.json()


OK 200


In [6]:
data

{'id': 133094316,
 'username': None,
 'resource_state': 2,
 'firstname': 'Xaver',
 'lastname': 'Heuser',
 'bio': '',
 'city': 'Troisdorf',
 'state': 'Nordrhein-Westfalen',
 'country': 'Germany',
 'sex': 'M',
 'premium': False,
 'summit': False,
 'created_at': '2024-03-10T13:27:59Z',
 'updated_at': '2025-08-01T13:07:19Z',
 'badge_type_id': 0,
 'weight': None,
 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/133094316/33608972/1/medium.jpg',
 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/133094316/33608972/1/large.jpg',
 'friend': None,
 'follower': None}

In [7]:
athlete_id = data['id']
print(f"Athlete ID: {athlete_id}")

Athlete ID: 133094316


# Athlete stats

In [8]:
athlete_stats_url = f'https://www.strava.com/api/v3/athletes/{athlete_id}/stats'
r = requests.get(athlete_stats_url, headers=headers, timeout=10)
r.raise_for_status()

In [9]:
athlete_stats = r.json()
athlete_stats

{'biggest_ride_distance': 100376.0,
 'biggest_climb_elevation_gain': 284.2,
 'recent_ride_totals': {'count': 3,
  'distance': 57342.200000000004,
  'moving_time': 7987,
  'elapsed_time': 8110,
  'elevation_gain': 491.0,
  'achievement_count': 0},
 'all_ride_totals': {'count': 85,
  'distance': 2048763.2999999993,
  'moving_time': 312603,
  'elapsed_time': 330536,
  'elevation_gain': 17977.3},
 'recent_run_totals': {'count': 18,
  'distance': 194724.3,
  'moving_time': 60273,
  'elapsed_time': 60696,
  'elevation_gain': 1296.0,
  'achievement_count': 0},
 'all_run_totals': {'count': 184,
  'distance': 1803229.1000000003,
  'moving_time': 550564,
  'elapsed_time': 555842,
  'elevation_gain': 12858.2},
 'recent_swim_totals': {'count': 0,
  'distance': 0,
  'moving_time': 0,
  'elapsed_time': 0,
  'elevation_gain': 0,
  'achievement_count': 0},
 'all_swim_totals': {'count': 6,
  'distance': 3017.5,
  'moving_time': 3462,
  'elapsed_time': 7108,
  'elevation_gain': 0.0},
 'ytd_ride_totals':

# Zones and Clubs

In [17]:
athlete_zones_url = f'https://www.strava.com/api/v3/athlete/clubs'
r = requests.get(athlete_zones_url, headers=headers, timeout=10)
r.raise_for_status()

In [18]:
zones = r.json()
zones

[]

# Gears